In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import helper_functions

In [2]:
mosquito_df = pd.read_csv('data_files/Mosquito_Trap_Data.csv')
weather_df = pd.read_csv('data_files/weather_stats_edmonton.csv')

In [3]:
mosquito_df_prep = helper_functions.clean_mosquito_df(mosquito_df)
weather_df_prep = helper_functions.clean_weather_df(weather_df)
weather_df_prep = helper_functions.preprocess_weather(weather_df_prep, ['2D', '7D', '14D'])



## Data Cleaning

### mosquito_df

1. Find and remove/impute missing values.
2. Remove rows marked as 'No' in 'Include' column
3. Remove rows marked as 'UnID' in the 'IDd' column
4. Aggregate data for species for the same dates - groupby
5. (Maybe) Aggregate data for locations since no locations exist in weather.


### weather_df

1. Remove columns: visibility, cloud cover, humidex, windchill, solar radiation
2. Find and remove/impute missing values.
3. Remove info from weather df for date ranges not found in Mousquito_df with a 2 week buffer to account for candles

## Data Pre-processing/Feature Engineering